### Script for video capture

In [2]:
filename = '/Users/vassiki/Desktop/brainhack/scannerEye/SamLocR3.mov'

def read_video(filename):
    import cv2

    cap = cv2.VideoCapture(filename)
    while not cap.isOpened():
        cap = cv2.VideoCapture(filename)
        cv2.waitKey(1000)
        print "Wait for the header"
    num_frames = cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)
    print "Total Frames: "+str(num_frames)
    pos_frame = cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES)

    # to return time of each frame in ms 
    time_frame = []
    data = []
    while True:
        flag, frame = cap.read()

        if flag:
            # The frame is ready and already captured
            cv2.imshow('video', frame)
            res = cv2.resize(frame,None,fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
            data.append(res)
            pos_frame = cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES)
            time = cap.get(cv2.cv.CV_CAP_PROP_POS_MSEC)
            time_frame.append(time)
        else:
            # The next frame is not ready, so we try to read it again
            cap.set(cv2.cv.CV_CAP_PROP_POS_FRAMES, pos_frame-1)
            print "frame is not ready"
            # It is better to wait for a while for the next frame to be ready
            cv2.waitKey(1000)
            time = cap.get(cv2.cv.CV_CAP_PROP_POS_MSEC)
            time_frame.append(time)
            #print "here?"

        if cv2.waitKey(10) == 27:
            break
        if cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES) == cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT):
            # If the number of captured frames is equal to the total number of frames,
            # we stop
            break

    return data, time_frame
    cv2.destroyAllWindows()  

In [ ]:
data, time_frame = read_video(filename) 

### Identify blink events in video

In [7]:
blinkfile = '/Users/vassiki/Desktop/brainhack/scannerEye/SamLocBlinks.txt'

def blinkFlag(blinkfile,time_frame):
    import numpy as np
    blinkInterval = []
    with open(blinktags) as f:
        lines = f.readlines()
        for line in lines:
            fields = line.split('\t')
            non_empty_fields = [f for f in fields if f != '']
            # convert string to float, then 
            if non_empty_fields[0] == 'blinks':
                blinkInterval.append([float(non_empty_fields[1])*1000,
                                  float(non_empty_fields[2])*1000])
            else:
                blinkInterval.append([0,0])


    # getting everything between the intervals that mark the blinks
    blinktimes = []
    for idx,blink in enumerate(blinkInterval):
        interval = np.arange(blinkInterval[idx][0],blinkInterval[idx][1],1.0)
        blinktimes.append(interval.tolist())
    
    blinks = [item for sublist in blinktimes for item in sublist]

    blink_tags = [0] * len(time_frame)

    timestamps = [t*0.001 for t in time_frame]

    for idx,stamp in enumerate(time_frame):
        #print int(stamp)
        if int(stamp) in blinks:
            blink_tags[idx] = 1
    
    return blink_tags

### SGD Classifier

In [12]:
def run_classifier(data,labels):
    
    from sklearn.linear_model import SGDClassifier
    
    X = data
    y = labels

    # collapsing rgb values
    x = np.asarray([f.mean(-1).reshape(-1) for f in data])

    # data folding-this needs to be optimized
    # right now, just leaving ~1000 frames to classify
    X_train, y_train = x[:7000], y[:7000]
    X_test, y_test = x[7000:], y[7000:]

    clf = SGDClassifier(loss="hinge", penalty="l2")
    clf.fit(X_train, y_train)  
    
    # predict of test data
    y_pred = clf.predict(X_test)
    
    from sklearn.metrics import classification_report

    pred = y_pred.tolist()
    true = y_test
    target_names = ['non-blink','blink']

    print(classification_report(true, pred, target_names=target_names))
    
    return clf

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

             precision    recall  f1-score   support

  non-blink       0.98      0.80      0.88       744
      blink       0.38      0.90      0.54       101

avg / total       0.91      0.82      0.84       845



### Scratchpad

In [26]:
filename = '/Users/vassiki/Desktop/brainhack/scannerEye/SamHemiR1.mov'

# function for new movie that returns new data and time frame, no annotations yet!!

# place function from first box here


Total Frames: 10213.0


In [28]:
x_new = np.asarray([f.mean(-1).reshape(-1) for f in newdata])

y_new_pred = clf.predict(x_new)

In [40]:
y_new_pred.tolist().count(0)

9174

In [ ]:
filename = '/Users/vassiki/Desktop/brainhack/scannerEye/SamHemiR1.mov'

import cv2

cap = cv2.VideoCapture(filename)
while not cap.isOpened():
    cap = cv2.VideoCapture(filename)
    cv2.waitKey(1000)
    print "Wait for the header"
num_frames = cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)
print "Total Frames: "+str(num_frames)
pos_frame = cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES)

# to return time of each frame in ms 
time_frame = []
preds = y_new_pred.tolist()
count = len(pred)

while True:
    flag, frame = cap.read()

    if flag:
        # The frame is ready and already captured
        #cv2.imshow('video', frame)
        res = cv2.resize(frame,None,fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
        cv2.imshow('video', res)
        pos_frame = cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES)
        time = cap.get(cv2.cv.CV_CAP_PROP_POS_MSEC)
        time_frame.append(time)
        #print str(time)+" s"
        #print str(pos_frame)+" frames"
        if count > 0:
            if preds[len(pred)-count] == 1:
                print "blink"
            else:
                print "eye"
            count = count-1
    else:
        # The next frame is not ready, so we try to read it again
        cap.set(cv2.cv.CV_CAP_PROP_POS_FRAMES, pos_frame-1)
        print "frame is not ready"
        # It is better to wait for a while for the next frame to be ready
        cv2.waitKey(1000)
        time = cap.get(cv2.cv.CV_CAP_PROP_POS_MSEC)
        time_frame.append(time)
        #print "here?"

    if cv2.waitKey(10) == 27:
        break
    if cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES) == cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT):
        # If the number of captured frames is equal to the total number of frames,
        # we stop
        break

cv2.destroyAllWindows()